In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from typing import Dict, List, Tuple

import numpy as np
from PIL import Image

from pysolotools.consumers import Solo
from pysolotools.converters.solo2coco import SOLO2COCOConverter
from pysolotools.core.models import KeypointAnnotationDefinition, RGBCameraCapture
from pysolotools.core.models import BoundingBox2DLabel, BoundingBox2DAnnotation
from pysolotools.core.models import BoundingBox3DLabel, BoundingBox3DAnnotation
from pysolotools.core.models import Frame, Capture

from scipy.spatial.transform import Rotation as R

from torchvision.datasets import ImageFolder
from torchvision.models import swin_v2_t, Swin_V2_T_Weights
# from torch.utils.data import ConcatDataset, DataLoader

import torch
from torch import nn

import matplotlib.pyplot as plt

import utils
# %matplotlib ipympl

c:\Users\LeonardNgo\miniconda3\envs\cvmpa\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3527.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [3]:
train_folder = 'D:/Unity/dataset/solo_3'
# training_dir = './data/train'
# testing_dir = './data/test'

In [97]:
solo = Solo(data_path=train_folder)


def pos_to_pixel(capture, pos) -> np.ndarray:
    r = R.from_quat(capture.rotation)
    translation = np.array(capture.position)

    camera_coordinates = r.inv().apply((pos - translation).T).T - translation
    pixel_coordinates = intrinsic @ camera_coordinates
    pixels = pixel_coordinates / pixel_coordinates[-1]
    # print()
    # return pixels[:2, 0].astype(np.int_)
    return pixels#.astype(np.int_)[:2]


for frame_idx, frame in enumerate(solo.frames()):
    # print(f'\r{frame_idx}/{len(solo.frames())}', end='')
    
    for i, capture in enumerate(frame.captures):
        camera = utils.Camera.from_unity(capture)

        resolution = np.array(capture.dimension)
        intrinsic = np.array([capture.matrix]).reshape((3, 3))
        # intrinsic /= intrinsic[-1,-1]
        # intrinsic[1, 1] = -intrinsic[1, 1]

        translation = np.array(capture.position)

        gt_pixels = {}
        bboxes = [anno for anno in capture.annotations if isinstance(anno, BoundingBox2DAnnotation)][0]
        for bbox in bboxes.values:
            pixel = np.array(bbox.origin) + np.array(bbox.dimension)/2
            gt_pixels[bbox.instanceId] = pixel.astype(np.int_)
            # print(bbox.instanceId, bbox.labelName)

        # for key, item in gt_pixels.items():
        #     print(key, item)

        r = R.from_quat(capture.rotation)
        pixels = {}
        anno_3d = [anno for anno in capture.annotations if isinstance(anno, BoundingBox3DAnnotation)][0]
        
        # relative coordinate convertion is correct (has been checked)
        poses = []
        for anno in anno_3d.values:
            t = np.array(anno.translation)
            # World coordinate = rorated camera coordinate + translation
            # P_w = R_w @ P_c + t_w
            pos = np.array(r.apply(t.T).T + translation)
            poses.append(pos)
            # print(pos)
        
        posses_ = [pos_to_pixel(capture, pos) for pos in poses]
        for idx, pos, gt_pos in zip([anno.instanceId for anno in anno_3d.values], camera(torch.tensor(poses), [*resolution, 1]), gt_pixels.values()):
        # for idx, pos, gt_pos in zip([anno.instanceId for anno in anno_3d.values], posses_, gt_pixels.values()):
            print(idx, np.sqrt(((pos.numpy() - gt_pos)**2).sum()), pos.numpy(), gt_pos)
            # print(idx, pos, gt_pos)
        
        for world_pos in poses:
            camera_pos = (world_pos - translation) @ r.inv().as_matrix()
            # print(camera_pos)

            # pixel_coordinates = camera_pos @ (intrinsic * (np.diag([-resolution[0], resolution[1], 1])/2)).T
            pixel_coordinates = intrinsic.dot(camera_pos)
            # pixel_coordinates = (intrinsic * (np.diag([resolution[0], resolution[1], 1])/2)).dot(camera_pos)
            # print(pixel_coordinates)
            if pixel_coordinates[-1] != 0:
                pixel_coordinates = pixel_coordinates / pixel_coordinates[-1]
            # print(pixel_coordinates)
            # pixel = pixel_coordinates[:2] + np.array(resolution)/2
            # print(pixel)
            print(
                np.array([
                    int(-(pixel_coordinates[0] * resolution[0]) / 2.0 + (resolution[0] * 0.5)),
                    int((pixel_coordinates[1] * resolution[1]) / 2.0 + (resolution[1] * 0.5)),
                ])
            )

        break
        print('-'*30)
    break



1 1.0193212469174924 [767.9947  220.16843] [769 220]
2 1.0140845578686535 [551.      220.16843] [550 220]
3 18.852831218877323 [551.          3.1737087] [550  22]
[767 220]
[551 220]
[551   3]


In [ ]:
def _project_pt_to_pixel_location(pt, projection, img_height, img_width):
    _pt = projection.dot(pt)

    # compute the perspective divide. Near clipping plane should take care of
    # divide by zero cases, but we will check to be sure

    if _pt[2] != 0:
        _pt /= _pt[2]

    return np.array([
        int(-(_pt[0] * img_width) / 2.0 + (img_width * 0.5)),
        int((_pt[1] * img_height) / 2.0 + (img_height * 0.5)),
    ])


In [81]:
intrinsic

array([[-0.91891291, -0.        , -0.        ],
       [-0.        ,  1.731012  , -0.        ],
       [-0.        , -0.        ,  1.        ]])

In [61]:
_intrinsic

array([[  -0.91891291,   -0.        ,   -0.        ],
       [  -0.        ,   -1.731012  ,   -0.        ],
       [-551.        ,  292.5       ,    1.        ]])

In [59]:
intrinsic

array([[-0.91891291, -0.        , -0.        ],
       [-0.        , -1.731012  , -0.        ],
       [-0.        , -0.        ,  1.        ]])

In [32]:
R.identity().from_quat([
    0.08282778, 0.901980042, 0.199964061, -0.373612136
]).as_euler('xyz', degrees=True)

array([155.00001243, -44.99997513, 179.99999927])

In [34]:
quat = R.from_euler('xyz', [-25, 225, 0], degrees=True).as_quat()
print(quat)
R.from_quat(quat).as_euler('xyz', degrees=True)

[ 0.08282785  0.9019799   0.19996413 -0.37361231]


array([155., -45., 180.])

In [30]:
R.from_euler('xyz', [-25, 225, 0], degrees=True).as_euler('xyz', degrees=True)

array([155., -45., 180.])

In [ ]:


        #     # camera_coordinates = r.inv().apply((pos - translation).T).T
        #     ueler = r.as_matrix()
        #     # print('pos', pos, pos - translation)
        #     camera_coordinates = (pos - translation) @ ueler
        #     # print(camera_coordinates)
        #     _intrinsic = intrinsic * np.array([-resolution[0]/2, resolution[1]/2, 1])
        #     # print(camera_coordinates.shape)
        #     pixel_coordinates = camera_coordinates @ _intrinsic
        #     # print(pixel_coordinates)
        #     # print(camera_coordinates.shape, intrinsic.shape)
        #     pixel = pixel_coordinates / pixel_coordinates[-1]
        #     pixel[:2] += resolution/2
        #     pixels[anno.instanceId] = pixel[:2].astype(np.int_)
        # poses = np.array(poses)
        # # print(poses.shape)
        # camera_coordinates = (poses - translation[np.newaxis, ...]) @ ueler
        # # print(camera_coordinates.shape)
        # pixel_coordinates = camera_coordinates @ _intrinsic
        # pixel = pixel_coordinates / pixel_coordinates[:, -1]
        # pixel[:, :2] += resolution/2
        # print(pixel[:, :2].astype(np.int_))